In [14]:
import re, nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('portuguese'))
wordnet_lemmatizer = WordNetLemmatizer()

def normalizer(tweet):
    only_letters = re.sub("[^a-zA-Z]", " ",tweet) 
    tokens = nltk.word_tokenize(only_letters)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas



In [15]:
tweets=pd.read_csv("/home/bruno/Documents/artigo/R/Dilma_oplexiconLimpo.txt",delimiter="\t",encoding='latin-1')


In [16]:
pd.set_option('display.max_colwidth', -1) # Setting this so we can see the full content of cells
tweets['normalized_tweet'] = tweets.text.apply(normalizer)
tweets[['text','normalized_tweet']].head()

,text,normalized_tweet
0,geraldo alckmin reeleito governador sao paulo ja ta decidido dilma vai pro segundo turno contra aecio neves aeciodevirada,"[reeleito, governador, sao, paulo, ja, ta, decidido, dilma, vai, pro, segundo, turno, contra, aecio, neve, aeciodevirada]"
1,dilma rousseff pt aecio neves psdb vao disputar turno,"[pt, aecio, neve, psdb, vao, disputar, turno]"
2,voces idolatrando aecio minimo nao sabem nada sobre governo so vao status odiar dilma parabens galera,"[aecio, minimo, nao, sabem, nada, sobre, governo, so, vao, status, odiar, dilma, parabens, galera]"
3,coisa nordestino votar dilma so causa bolsa familia mentira aqui trabalhador gentr q acorda cedo pra trabalha,"[votar, dilma, so, causa, bolsa, familia, mentira, aqui, trabalhador, gentr, q, acorda, cedo, pra, trabalha]"
4,unica coisa n queria alckmin governador dilma aceito,"[n, queria, alckmin, governador, dilma, aceito]"


In [17]:
from nltk import ngrams
def ngrams(input_list):
    #onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams+trigrams
tweets['grams'] = tweets.normalized_tweet.apply(ngrams)
tweets[['grams']].head()

,grams
0,"[reeleito governador, governador sao, sao paulo, paulo ja, ja ta, ta decidido, decidido dilma, dilma vai, vai pro, pro segundo, segundo turno, turno contra, contra aecio, aecio neve, neve aeciodevirada, reeleito governador sao, governador sao paulo, sao paulo ja, paulo ja ta, ja ta decidido, ta decidido dilma, decidido dilma vai, dilma vai pro, vai pro segundo, pro segundo turno, segundo turno contra, turno contra aecio, contra aecio neve, aecio neve aeciodevirada]"
1,"[pt aecio, aecio neve, neve psdb, psdb vao, vao disputar, disputar turno, pt aecio neve, aecio neve psdb, neve psdb vao, psdb vao disputar, vao disputar turno]"
2,"[aecio minimo, minimo nao, nao sabem, sabem nada, nada sobre, sobre governo, governo so, so vao, vao status, status odiar, odiar dilma, dilma parabens, parabens galera, aecio minimo nao, minimo nao sabem, nao sabem nada, sabem nada sobre, nada sobre governo, sobre governo so, governo so vao, so vao status, vao status odiar, status odiar dilma, odiar dilma parabens, dilma parabens galera]"
3,"[votar dilma, dilma so, so causa, causa bolsa, bolsa familia, familia mentira, mentira aqui, aqui trabalhador, trabalhador gentr, gentr q, q acorda, acorda cedo, cedo pra, pra trabalha, votar dilma so, dilma so causa, so causa bolsa, causa bolsa familia, bolsa familia mentira, familia mentira aqui, mentira aqui trabalhador, aqui trabalhador gentr, trabalhador gentr q, gentr q acorda, q acorda cedo, acorda cedo pra, cedo pra trabalha]"
4,"[n queria, queria alckmin, alckmin governador, governador dilma, dilma aceito, n queria alckmin, queria alckmin governador, alckmin governador dilma, governador dilma aceito]"


In [18]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

In [19]:
tweets[(tweets.sentimento == 'negativo')][['grams']].apply(count_words)['grams'].most_common(20)

[('votar dilma', 3783),
 ('segundo turno', 1834),
 ('dilma aecio', 1595),
 ('ano passado', 969),
 ('pra votar', 839),
 ('vota dilma', 629),
 ('aecio dilma', 581),
 ('dilma nao', 579),
 ('pra votar dilma', 575),
 ('povo burro', 552),
 ('votar aecio', 536),
 ('bolsa familia', 531),
 ('pra dilma', 491),
 ('votou dilma', 440),
 ('dilma novo', 418),
 ('pior dilma', 405),
 ('dilma vai', 403),
 ('vai ser', 398),
 ('vai votar', 362),
 ('votar nela', 347)]

In [20]:
tweets[(tweets.sentimento == 'positivo')][['grams']].apply(count_words)['grams'].most_common(20)

[('dilma aecio', 1040),
 ('segundo turno', 761),
 ('aecio dilma', 463),
 ('sao paulo', 401),
 ('aecio marina', 374),
 ('dilma marina', 345),
 ('dilma nao', 328),
 ('melhor dilma', 311),
 ('urnas apuradas', 297),
 ('apoiar dilma', 251),
 ('pra dilma', 251),
 ('dilma aecio marina', 232),
 ('lugar urnas', 228),
 ('dilma lugar', 225),
 ('dilma lugar urnas', 225),
 ('lugar urnas apuradas', 225),
 ('dilma sao', 217),
 ('bolsa familia', 213),
 ('vota dilma', 195),
 ('dilma vai', 181)]

In [21]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [22]:
vectorized_data = count_vectorizer.fit_transform(tweets.text)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [25]:
def sentiment2target(sentiment):
    return {
        'negativo': -1,
        'neutro': 0,
        'positivo' : 1
    }[sentiment]
targets = tweets.sentimento.apply(sentiment2target)

In [27]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf_output = clf.fit(data_train, targets_train)

In [ ]:
clf.score(data_test, targets_test)